In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

URL="https://www.redbus.in/online-booking/astc/?utm_source=rtchometile"
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME,'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  
              
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  
            
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  
            
            bus_name_elements = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
            bus_type_elements = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
            departing_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
            duration_elements = driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
            arrival_elements= driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
            rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            fare_elements = driver.find_elements(By.CSS_SELECTOR,"span.f-19.f-bold")
            seat_availability_elements = driver.find_elements(By.CSS_SELECTOR,".seat-left")
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Arrival": arrival_elements[i].text,
                    "Rating": rating_elements[i].text if i < len(rating_elements) else '0',
                    "Fare": fare_elements[i].text,
                    "Seat_Available": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

all_bus_details = []

def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs ')][text()='{page}']")))
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5) 
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
           
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            
         
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

scrape_all_pages()

df = pd.DataFrame(all_bus_details)

df.to_csv('Assam.csv', index=False)

driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/north-lakhimpur-to-sibsagar: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7B0086CB5+28821]
	(No symbol) [0x00007FF7AFFF3840]
	(No symbol) [0x00007FF7AFE9578A]
	(No symbol) [0x00007FF7AFEE91BE]
	(No symbol) [0x00007FF7AFEE94AC]
	(No symbol) [0x00007FF7AFF32647]
	(No symbol) [0x00007FF7AFF0F33F]
	(No symbol) [0x00007FF7AFF2F412]
	(No symbol) [0x00007FF7AFF0F0A3]
	(No symbol) [0x00007FF7AFEDA778]
	(No symbol) [0x00007FF7AFEDB8E1]
	GetHandleVerifier [0x00007FF7B03BFCAD+3408013]
	GetHandleVerifier [0x00007FF7B03D741F+3504127]
	GetHandleVerifier [0x00007FF7B03CB5FD+3455453]
	GetHandleVerifier [0x00007FF7B014BDBB+835995]
	(No symbol) [0x00007FF7AFFFEB5F]
	(No symbol) [0x00007FF7AFFFA814]
	(No symbol) [0x00007FF7AFFFA9AD]
	(No symbol) [0x00007FF7AFFEA199]
	BaseThreadInitThunk [0x00007FFDEEED7374+20]
	RtlUserThreadStart [0x00007FFDEFDFCC91+33]

Error occurred while scraping bus details for https://www.redbus.i

NameError: name 'driver' is not defined

In [3]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Arrival,Rating,Fare,Seat_Available
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 170878,Bharat Benz A/C Seater (2+2),10:35,04h 40m,15:15,4.4,298,5 Seats available
1,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 177482,Tata A/C Seater (2+2),14:00,05h 00m,19:00,2.3,298,10 Seats available
2,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 139847,Volvo AC Seater 2+2,16:15,04h 15m,20:30,4.1,298,4 Seats available
3,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Pranjit Travels (Under ASTC),NON A/C Seater (2+1),10:30,05h 15m,15:45,3.8,450,30 Seats available
4,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Chartered Bus - ASTC,A/C Seater Push Back (2+2),11:30,04h 45m,16:15,4.4,397.08,9 Seats available
...,...,...,...,...,...,...,...,...,...,...
317,North Lakhimpur to Nagaon (Assam),https://www.redbus.in/bus-tickets/north-lakhim...,Yatra Travels,NON A/C Seater (2+1),20:40,06h 20m,03:00,4.1,540,23 Seats available
318,North Lakhimpur to Nagaon (Assam),https://www.redbus.in/bus-tickets/north-lakhim...,Yatra Travels,A/C Seater / Sleeper (2+1),21:00,06h 30m,03:30,4.1,585,28 Seats available
319,North Lakhimpur to Nagaon (Assam),https://www.redbus.in/bus-tickets/north-lakhim...,PARIJAT TRAVELS(UNDER ASTC),NON A/C Seater (2+1),20:56,06h 04m,03:00,3.9,590,19 Seats available
320,North Lakhimpur to Nagaon (Assam),https://www.redbus.in/bus-tickets/north-lakhim...,DESTINY TRAVELS,NON AC Seater / Sleeper 2+1,18:30,07h 00m,01:30,1.7,498.75,35 Seats available
